<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Generalized Eigenvalue Computation</strong>
</div></div>


# Notebook 2 — Algorithms: Computing Generalized Eigenvalues (GEP)

**Objective.** Present stable, practical methods to compute generalized eigenvalues and eigenvectors for the pencil $(A,B)$:  
- symmetric-definite reduction via **Cholesky whitening**,  
- the **QZ algorithm** (generalized Schur form) for general regular pencils,  
- how to **read eigenvalues** from Schur form,  
- how to **audit residuals** and understand **backward stability**, and  
- how to **reorder** to extract **deflating subspaces**.

Conceptual foundations and variational viewpoints are covered in Notebook 1.



## 1. Why not $\qquad \det(A-\lambda B)=0$ or $B^{-1}A$?

- Forming $\det(A-\lambda B)$ and finding its roots is **ill-conditioned** and scales poorly ($\mathcal{O}(n^3)$ per evaluation; root finding is unstable for high-degree polynomials).  
- Forming $B^{-1}A$ explicitly is **dangerous** when $B$ is ill-conditioned/singular; it also destroys structure.  
- Modern solvers avoid explicit inverses and use **orthogonal/unitary** transformations to reach a **generalized Schur form**.


In [ ]:

import numpy as np

np.set_printoptions(precision=6, suppress=True)

try:
    from scipy.linalg import cholesky, eigh, qz, ordqz
    HAVE_SCIPY = True
except Exception as e:
    HAVE_SCIPY = False
    print("SciPy not available; QZ and ordqz demos will be skipped:", e)

def residual(A,B,lam,x):
    num = np.linalg.norm(A@x - lam*(B@x))
    den = (np.linalg.norm(A)*np.linalg.norm(x) + 1e-15)
    return num/den



## 2. Symmetric-definite reduction (when $A=A^\top$, $B=B^\top \succ 0$)

If $A=A^\top$ and $B=B^\top \succ 0$, we can **whiten** $B$ using its Cholesky factor $B=L L^\top$ and solve a **standard symmetric** eigenproblem.

**Whitening.** Let $z=L^\top x$ and define $\tilde A = L^{-1} A L^{-\top}$. Then  
$\qquad A x = \lambda B x \iff \tilde A z = \lambda z.$

**Algorithm.**  
1) Compute $L$ (Cholesky of $B$).  
2) Form $\tilde A = L^{-1} A L^{-\top}$ by triangular solves.  
3) Compute eigenpairs $(\lambda, z)$ of $\tilde A$ (symmetric).  
4) Recover $x = L^{-\top} z$; optionally $B$-normalize: $x \leftarrow x/\sqrt{x^\top B x}$.


In [ ]:

def gep_symmetric_definite(A,B):
    """Solve A x = λ B x for symmetric A and SPD B via Cholesky whitening.
    Returns (lam, X) with columns of X being B-normalized eigenvectors.
    """
    if not HAVE_SCIPY:
        raise RuntimeError("SciPy required here for robust Cholesky/eigh.")
    L = cholesky(B, lower=True)
    # Solve for At = L^{-1} A L^{-T}
    At = np.linalg.solve(L, A)
    At = At @ np.linalg.solve(L.T, np.eye(A.shape[0]))
    lam, Z = eigh(At)
    # Map back and B-normalize
    X = np.linalg.solve(L.T, Z)
    for i in range(X.shape[1]):
        xi = X[:,i]
        nB = np.sqrt(xi.T @ B @ xi)
        if nB > 0:
            X[:,i] = xi / nB
    return lam, X

# Demo
np.random.seed(0)
n = 5
R = np.random.randn(n,n)
B = R@R.T + n*np.eye(n)       # SPD
S = np.random.randn(n,n); A = 0.5*(S+S.T)

lam, X = gep_symmetric_definite(A,B)
print("Eigenvalues (symmetric-definite route):", lam)
print("||X^T B X - I||:", np.linalg.norm(X.T @ B @ X - np.eye(n)))
print("||A X - B X diag(lam)||:", np.linalg.norm(A@X - B@X@np.diag(lam)))



## 3. General regular pencils: the QZ algorithm (generalized Schur form)

For arbitrary regular $(A,B)$, compute unitary $Q,Z$ such that  
$\qquad Q^\top A Z = S, \quad Q^\top B Z = T,$  
with $S,T$ (quasi)upper-triangular. This is the **generalized Schur form**.

- **Eigenvalues** are read from diagonal blocks as ratios $\qquad \lambda_i = S_{ii}/T_{ii}$ (with $T_{ii}=0$ encoding $\lambda_i=\infty$).  
- The orthogonal/unitary steps make QZ **backward stable**: computed results are exact for a nearby $(A+\Delta A,B+\Delta B)$ with small perturbations.


In [ ]:

def qz_eigs(A,B,output='real'):
    if not HAVE_SCIPY:
        raise RuntimeError("SciPy required for QZ.")
    S, T, Q, Z = qz(A, B, output=output)
    alpha = np.diag(S).astype(complex)
    beta  = np.diag(T).astype(complex)
    w = alpha / beta
    return w, (S,T,Q,Z)

# Small demo with a nonsymmetric pair
A = np.array([[4., 2., 0.],
              [1., 3., 1.],
              [0., 0., 2.]])
B = np.array([[1., 0., 0.],
              [0., 2., 0.],
              [0., 0., 1.]])

if HAVE_SCIPY:
    w, (S,T,Q,Z) = qz_eigs(A,B,output='real')
    print("QZ eigenvalues (ratios S_ii/T_ii):", w)
    # Residual check with right Schur vectors (columns of Z span invariant subspaces)
    for i in range(len(w)):
        x = Z[:,i]
        print(f"i={i}, residual:", residual(A,B,w[i],x))
else:
    print("SciPy not available; skipping QZ demo.")



## 4. Detecting $\lambda=\infty$ in practice

In QZ form, $T_{ii}\approx 0$ indicates $\lambda_i=\infty$ (since $\lambda_i=S_{ii}/T_{ii}$). Equivalently, flip to the **reciprocal pencil**
$\qquad B - \mu A $
and look for eigenvalues at $\mu=0$.


In [ ]:

# Example: regular pencil with B singular -> one eigenvalue at infinity
A = np.array([[1., 0.],
              [0., 2.]])
B = np.array([[1., 0.],
              [0., 0.]])

if HAVE_SCIPY:
    w1, (S1,T1,_,Z1) = qz_eigs(A,B,output='real')
    print("Eigenvalues of (A,B):", w1, "  (one should be ∞)")
    print("T diag:", np.diag(T1))  # one ~0
    # Reciprocal
    w2, _ = qz_eigs(B,A,output='real')
    print("Eigenvalues of reciprocal (B,A):", w2, "  (expect a zero)")
else:
    print("SciPy not available; ∞-eigenvalue demo skipped.")



## 5. Reordering and deflating subspaces (via `ordqz`)

To extract a **deflating subspace** for a spectral region (e.g., $|\lambda|<1$ for discrete-time stability), reorder the generalized Schur form so that the desired eigenvalues appear first. The corresponding first columns of $Z$ span the deflating subspace.


In [ ]:

if HAVE_SCIPY:
    # Discrete-time example: select |lambda| < 1
    A = np.array([[0.8, 0.3, 0.0],
                  [0.0, 1.2, 0.2],
                  [0.0, 0.0, 0.9]])
    B = np.eye(3)

    def select_inside(alpha, beta):
        w = alpha/beta
        return np.abs(w) < 1.0

    S, T, Q, Z, alpha, beta, sdim = ordqz(A, B, sort=select_inside, output='real')
    w = alpha/beta
    print("Reordered eigenvalues:", w)
    print("Count inside unit disk:", sdim)
    # Extract deflating subspace U = Z[:, :sdim] and verify approximate invariance
    U = Z[:, :sdim]
    PA = U @ np.linalg.lstsq(U, (A@U), rcond=None)[0]
    PB = U @ np.linalg.lstsq(U, (B@U), rcond=None)[0]
    print("||A U - Proj_U(A U)||:", np.linalg.norm(A@U - PA))
    print("||B U - Proj_U(B U)||:", np.linalg.norm(B@U - PB))
else:
    print("SciPy not available; ordqz demo skipped.")



## 6. Backward error viewpoint and residual auditing

A computed pair $(\hat\lambda, \hat x)$ satisfies $A\hat x - \hat\lambda B \hat x = r$. With QZ (orthogonal/unitary updates), the algorithm is **backward stable**: it computes the exact Schur form of a nearby pencil $(A+\Delta A, B+\Delta B)$ with small perturbations.

**Practical check.** Report **normalized residuals**  
$\qquad \dfrac{\|A\hat x - \hat\lambda B\hat x\|}{\|A\|\,\|\hat x\|}$  
for all computed modes; small residuals give confidence in the solution.


In [ ]:

# Residual audit on a random example
np.random.seed(7)
A = np.random.randn(6,6)
B = np.random.randn(6,6)

if HAVE_SCIPY:
    w, (S,T,Q,Z) = qz_eigs(A,B,output='complex')
    res = np.array([residual(A,B,w[i],Z[:,i]) for i in range(len(w))])
    print("Median normalized residual:", np.median(res))
    print("Max normalized residual   :", np.max(res))
else:
    print("SciPy not available; residual audit skipped.")



## 7. Practical recipes and pitfalls

- **If $A=A^\top$, $B=B^\top \succ 0$**: use the **Cholesky route** (symmetric-definite). You get real eigenvalues and $B$-orthonormal eigenvectors.  
- **Otherwise (regular pencils)**: use **QZ**; read eigenvalues from $S_{ii}/T_{ii}$, detect $\lambda=\infty$ via $T_{ii}\approx 0$.  
- **Never form $B^{-1}A$ explicitly**; prefer triangular solves or Schur/QZ.  
- **Scale/equilibrate** poorly scaled inputs before solving (see Notebook 3).  
- **Always check residuals**; when in doubt, verify with multiple methods (e.g., Cholesky vs QZ) in cases where both apply.



## 8. Exercises

1) **Symmetric-definite route.** Create random SPD $B$ and symmetric $A$; solve via whitening and verify $X^\top B X \approx I$ and $AX \approx BX\Lambda$.  
2) **QZ reading.** Construct a pair with a singular $B$; use QZ to detect $T_{ii}\approx 0$ and confirm the presence of $\lambda=\infty$ using the reciprocal pencil.  
3) **Reordering.** For a discrete-time model $(A,E)$, use `ordqz` to separate $|\lambda|<1$ modes and verify the deflating subspace property numerically.  
4) **Backward error.** For a random pencil, compute all residuals and relate their magnitudes to problem scaling (try equilibrating $A,B$ first).  
5) **Structure check.** For the symmetric-definite case, compare eigenvalues from the Cholesky route with those from QZ (they should agree to roundoff).
